In [ ]:
import os
os.environ["bilby_style"] = "none"


import bilby
import corner
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import seaborn as sns
from scipy import stats

from thesis_utils.gw import get_cbc_parameter_labels
from thesis_utils.plotting import (
    set_plotting, save_figure,
    get_default_figsize,
    get_default_corner_kwargs,
    plot_acceptance,
    get_corner_figsize,
    plot_multiple_bilby,
)
from thesis_utils.io import load_pickle
import thesis_utils.colours as thesis_colours

set_plotting()

In [ ]:
result_marg = bilby.core.result.read_in_result("outdir/paper_default_phase_marg_result.json")

In [ ]:
ns_marg = load_pickle("outdir/paper_default_phase_marg_nessai/nested_sampler_resume.pkl")

In [ ]:

result = bilby.core.result.read_in_result("outdir/paper_default_no_phase_marg_result.json")

In [ ]:
ns = load_pickle("outdir/paper_default_no_phase_marg_nessai/nested_sampler_resume.pkl")

In [ ]:
figsize = get_default_figsize()
figsize[1] *= 1.3
fig = plot_acceptance([ns_marg, ns], figsize=figsize)
save_figure(fig, "phase_baseline_diagnostics")

In [ ]:
corner_kwargs = get_default_corner_kwargs()
corner_kwargs.pop("fill_contours")
corner_kwargs["show_titles"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs.pop("color")

parameters = ["chirp_mass", "mass_ratio", "a_1", "a_2", "ra", "dec", "theta_jn", "phase", "psi"]
colours=[thesis_colours.teal, thesis_colours.yellow]
labels=["With phase marg.", "Without phase marg."]

fig = plot_multiple_bilby(
    [result_marg, result,],
    parameters=parameters,
    labels=labels,
    colours=colours,
    corner_labels=get_cbc_parameter_labels(parameters, units=True),
    fig=plt.figure(figsize=0.85 * get_corner_figsize(len(parameters))),
    add_legend=False,
    labelpad=0.1,
    **corner_kwargs,
)

handles = []
for l, c in zip(labels, colours):
    handles.append(
        Line2D([0], [0], color=c, label=l)
    )

fig.legend(
    handles=handles,
    loc="center",
    fontsize=14,
    bbox_to_anchor=(0.75, 0.75)
)
# axes = fig.get_axes()
# ndim = int(np.sqrt(len(axes)))
# axes[ndim - 1].get_legend().remove()
save_figure(fig, "phase_baseline_corner", "figures")

## Insertion indices

In [ ]:
indices_marg = np.array(ns_marg.insertion_indices)
indices_wo_marg = np.array(ns.insertion_indices)

In [ ]:
indices = [indices_marg, indices_wo_marg]

In [ ]:
linestyles = {
    "low_spin": "--",
    "high_spin": "-",
}

nlive = 2000
bins = np.arange(0, nlive+1, 20)
nbins = len(bins)

x = np.arange(0, nlive, 1)
analytic = x / x[-1]

figsize = get_default_figsize()
figsize[1] *= 1.2
fig, axs = plt.subplots(2, 2, figsize=figsize, sharex=True, sharey="col")

for j, idx in enumerate(indices):

    n = len(idx)
    axs[j, 0].hist(idx, bins=bins, density=True, histtype="step", color=f"C{j}")
    axs[j, 0].axhline(1 / nlive, zorder=-1, color="grey")
    axs[j, 0].set_ylabel("Density")
    
    _, counts = np.unique(idx, return_counts=True)
    estimated = np.cumsum(counts) / len(idx)
    axs[j, 1].plot(analytic - estimated, ls="-", c=f"C{j}")

    for ci in [0.5, 0.95, 0.997]:
        bound = (1 - ci) / 2
        bound_values = stats.binom.ppf(1 - bound, n, analytic) / n
        lower = (bound_values - analytic)
        upper = (analytic - bound_values)
        upper[0] = 0
        upper[-1] = 0
        lower[0] = 0
        lower[-1] = 0

        axs[j, 1].fill_between(x, lower, upper, color="grey", alpha=0.2)

    axs[j, 1].set_xlim(0, nlive-1)
    axs[j, 1].set_ylabel("Analytic CMF -\n Empirical CMF")


axs[-1, 0].set_xlabel("Insertion index")
axs[-1, 1].set_xlabel("Insertion index")

axs[0, 0].set_title(r"\textbf{With phase marg.}", loc="left", x=-0.3, pad=12)
axs[1, 0].set_title(r"\textbf{Without phase marg.}", loc="left", x=-0.3, pad=12)


plt.tight_layout()

save_figure(fig, "phase_insertion_indices")